<a href="https://colab.research.google.com/github/thravt/FederatedAggregation/blob/main/CNNDist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def makecnn():
  model = tf.keras.models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))
  return model

In [ ]:
import matplotlib.pyplot as plt
import PIL
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
modelC = makecnn()
modellist = []
for i in range(5):
  modellist.append(makecnn())
  modellist[i].set_weights(list(modelC.get_weights()))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Getting splits:
imagelist = np.array_split(train_images, 5)
labellist = np.array_split(train_labels, 5)

In [ ]:
for i in range(5):
  modellist[i].compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

In [ ]:
modelC.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

In [ ]:
for i in range(5):
  modellist[i].fit(imagelist[i], labellist[i], epochs = 25)

Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.1727 - f1_score: 0.1947 - loss: 5.0043
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.3668 - f1_score: 0.1954 - loss: 1.7245
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.4277 - f1_score: 0.1954 - loss: 1.5568
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.4893 - f1_score: 0.1955 - loss: 1.4137
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.5286 - f1_score: 0.1955 - loss: 1.3131
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5762 - f1_score: 0.1955 - loss: 1.1820
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.6225 - f1_score: 0.1955 - loss: 1.0748
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.6432 - f1_score: 0.1955 - loss: 0.9938
Epoch 9/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.6825 - f1_score: 0.1956 - loss: 0.9026
Epoch 10/25
313/313 ━━━━━━━━━━━━━━━━━

In [ ]:
client_preds = []
for model in modellist:
        preds = model.predict(test_images)
        client_preds.append(preds)
aggregated_preds = np.mean(np.array(client_preds), axis=0)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [ ]:
aggregated_preds = np.argmax(aggregated_preds, axis=1)

In [ ]:
aggregated_preds = aggregated_preds.reshape(-1, 1)

In [ ]:
modelC.fit(test_images, aggregated_preds, epochs = 25)

Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.2239 - f1_score: 0.1948 - loss: 5.0210
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.4593 - f1_score: 0.1950 - loss: 1.4962
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.5212 - f1_score: 0.1953 - loss: 1.2912
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - accuracy: 0.5663 - f1_score: 0.1953 - loss: 1.1654
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 44ms/step - accuracy: 0.6149 - f1_score: 0.1953 - loss: 1.0567
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.6434 - f1_score: 0.1954 - loss: 0.9568
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.6959 - f1_score: 0.1954 - loss: 0.8261
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.7166 - f1_score: 0.1954 - loss: 0.7631
Epoch 9/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.7461 - f1_score: 0.1954 - loss: 0.6831
Epoch 10/25
313/313 ━━━━━━━━━━━━━━━━━

In [ ]:
modelC.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.5956 - f1_score: 0.1956 - loss: 4.4741


[4.544413089752197,
 0.5949000120162964,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.11812098, 0.12130692, 0.11801121, 0.14404432, 0.14169686,
        0.2609953 , 0.2289887 , 0.25070158, 0.2887252 , 0.28330064],
       dtype=float32)>]